In [11]:
import nltk # using textBlob and tensoflow (CNN and LSTM )
import pprint
import tensorflow as tf
import pandas as pd
import numpy as np
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
# some source for this dic 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
data_train = pd.read_csv('C:/Project/movie_reviews/movie_reviews.csv')
data_test = pd.read_csv('C:/Project/movie_reviews./test_data.csv')
data_train['label'].value_counts()
sentiment_label = data_train['label'].values
tweet = data_train.text.values
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)

tokenizer.fit_on_texts(tweet)
encoded_docs = tokenizer.texts_to_sequences(tweet)
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_sequence = pad_sequences(encoded_docs, maxlen=200)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

vocab_size = 5000

embedding_vector_length = 32
from tensorflow.keras.layers import Bidirectional, LSTM

# Define the BiLSTM model
model_bilstm = Sequential()
model_bilstm.add(Embedding(vocab_size, embedding_vector_length, input_length=200))
model_bilstm.add(Bidirectional(LSTM(50, dropout=0.5, recurrent_dropout=0.5)))
model_bilstm.add(Dense(64, activation='relu'))
model_bilstm.add(Dropout(0.2))
model_bilstm.add(Dense(1, activation='sigmoid'))
model_bilstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model_bilstm.summary())

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mayti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 32)           160000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 100)               33200     
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 64)                6464      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 199729 (780.19 KB)
Trainable params: 199729 (780.19 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [2]:
history = model_bilstm.fit(padded_sequence,sentiment_label,validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5


1000/1000 [==============================] - 93s 90ms/step - loss: 0.4242 - accuracy: 0.7959 - val_loss: 0.4025 - val_accuracy: 0.8455
Epoch 2/5
1000/1000 [==============================] - 84s 84ms/step - loss: 0.2869 - accuracy: 0.8827 - val_loss: 0.2922 - val_accuracy: 0.8819
Epoch 3/5
1000/1000 [==============================] - 83s 83ms/step - loss: 0.2625 - accuracy: 0.8940 - val_loss: 0.3055 - val_accuracy: 0.8783
Epoch 4/5
1000/1000 [==============================] - 83s 83ms/step - loss: 0.2370 - accuracy: 0.9049 - val_loss: 0.3249 - val_accuracy: 0.8765
Epoch 5/5
1000/1000 [==============================] - 83s 83ms/step - loss: 0.2203 - accuracy: 0.9124 - val_loss: 0.3668 - val_accuracy: 0.8730


In [4]:
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=200)
    prediction = int(model_bilstm.predict(tw).round().item())
    print("Predicted label: ", abs(sentiment_label[prediction] - 1))
    return abs(sentiment_label[prediction] - 1)

data_test['Predicted_Sentiment'] = data_test['text'].apply(predict_sentiment)

1/1 [==============================] - 0s 321ms/step
Predicted label:  0
1/1 [==============================] - 0s 36ms/step
Predicted label:  1
1/1 [==============================] - 0s 36ms/step
Predicted label:  0
1/1 [==============================] - 0s 35ms/step
Predicted label:  1
1/1 [==============================] - 0s 36ms/step
Predicted label:  1
1/1 [==============================] - 0s 36ms/step
Predicted label:  1
1/1 [==============================] - 0s 34ms/step
Predicted label:  1
1/1 [==============================] - 0s 35ms/step
Predicted label:  1
1/1 [==============================] - 0s 31ms/step
Predicted label:  1
1/1 [==============================] - 0s 85ms/step
Predicted label:  1
1/1 [==============================] - 0s 36ms/step
Predicted label:  0
1/1 [==============================] - 0s 36ms/step
Predicted label:  0
1/1 [==============================] - 0s 29ms/step
Predicted label:  0
1/1 [==============================] - 0s 36ms/step
Predicted l

In [8]:
submission = data_test.copy()
submission

,Id,text,Predicted_Sentiment
0,0,What can possibly said about this movie other ...,0
1,1,I dont care how many bad reviews purple rain g...,1
2,2,Ken Russell directed this weird Not very ero...,0
3,3,This is a great movie from the lost age of rea...,1
4,4,I have a problem with the movie snobs who cons...,1
...,...,...,...
9995,9995,I first saw this on Thames television and love...,1
9996,9996,There are rumours that a fourth Underworld is ...,0
9997,9997,Jag är nyfiken Yellow is a lot of fun Like at...,1
9998,9998,A widely unknown strange little western with m...,1


In [9]:
submission = submission.drop(['text'], axis=1)
submission.to_csv('submission1.csv', header=['Id', 'Category'], index=None)